In [ ]:
!pip install PyTorch
!pip install TensorFlow

  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for PyTorch
  Running setup.py clean for PyTorch
Failed to build PyTorch
ERROR: Could not build wheels for PyTorch, which is required to install pyproject.toml-based projects


In [ ]:
import pandas as pd
from random import choice, seed
from transformers import pipeline, CamembertTokenizer, CamembertForMaskedLM
import torch

In [ ]:
!pip install sentencepiece

In [ ]:
# Set a random seed for reproducibility
seed(42)

# Load the CamemBERT tokenizer and model
tokenizer = CamembertTokenizer.from_pretrained('camembert-base')
model = CamembertForMaskedLM.from_pretrained('camembert-base')


sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForMaskedLM: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing CamembertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Check and use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
model.to(device)

# Initialize the fill-mask pipeline with the GPU device
fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer, device=0)  # device=0 for the first GPU

# Function to replace a random word in a sentence with a mask token
def mask_random_word(sentence):
    tokens = sentence.split()
    mask_position = choice(range(len(tokens)))
    tokens[mask_position] = tokenizer.mask_token
    return " ".join(tokens), tokens[mask_position]

# Function to get predictions for the masked sentence
def get_predictions(masked_sentence, original_word):
    predictions = fill_mask(masked_sentence)
    # Filter out the original word and return the rest
    synonyms = [pred['token_str'] for pred in predictions if pred['token_str'] != original_word]
    return synonyms


Using device: cuda


In [ ]:
!git clone https://github.com/melvin2504/Detecting-the-difficulty-level-of-French-texts/ project_repo

Cloning into 'project_repo'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 53 (delta 19), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (53/53), 1.19 MiB | 2.34 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
# Load the dataset
df = pd.read_csv("/content/project_repo/data/training_data.csv")

# Augment the dataset
augmented_sentences = []
augmented_labels = []
# Specify the difficulties to include
selected_difficulties = ['A2', 'B1', 'B2', 'C1']

for _, row in df.iterrows():
    sentence, difficulty = row['sentence'], row['difficulty']

    # Check if the difficulty is one of the selected ones
    if difficulty in selected_difficulties:
        masked_sentence, original_word = mask_random_word(sentence)
        predictions = get_predictions(masked_sentence, original_word)

        for synonym in predictions:
            new_sentence = masked_sentence.replace(tokenizer.mask_token, synonym)
            augmented_sentences.append(new_sentence)
            augmented_labels.append(difficulty)

# Create a DataFrame with the augmented data
augmented_df = pd.DataFrame({
    'sentence': augmented_sentences,
    'difficulty': augmented_labels
})




/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [ ]:
# Combine with the original dataset
augmented_df = pd.concat([df, augmented_df]).reset_index(drop=True)

# Save the augmented dataset
augmented_df.to_csv('augmented_training_data_3.csv', index=False)
